In [ ]:
os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "12345678"

graph = Neo4jGraph(refresh_schema=False)    

In [ ]:
# Hàm đếm tokens
def num_tokens_from_string(string: str, model_name: str = "gpt-3.5-turbo") -> int:
    encoding = tiktoken.encoding_for_model(model_name)
    return len(encoding.encode(string))

In [ ]:
import os
from concurrent.futures import ThreadPoolExecutor, as_completed

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

import tiktoken
from pydantic import BaseModel, Field

from typing import List, Optional

from neo4j import GraphDatabase
from graphdatascience import GraphDataScience

from langchain_neo4j import Neo4jGraph
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_community.vectorstores.neo4j_vector import (
    Neo4jVector,
    remove_lucene_chars,
)

from neo4j import Result
from typing import Dict, Any

In [ ]:
os.environ["OPENAI_API_KEY"] = "API-KEY-HERE"

print(os.getenv("OPENAI_API_KEY"))


In [ ]:
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

llm_transformer = LLMGraphTransformer(
  llm=llm, 
  node_properties=["description"],
  relationship_properties=["description"]
)

def process_text(text: str) -> List[Document]:
    doc = Document(page_content=text)
    return llm_transformer.convert_to_graph_documents([doc])

In [ ]:
# test
sample_text = ""
with open("data/sample_text.txt", "r", encoding="utf-8") as f:
    sample_text = f.read()

sample_doc = process_text(text=sample_text)

print(sample_doc)

In [ ]:
graph.add_graph_documents(
    sample_doc,
    baseEntityLabel=True,
    include_source=True
)

In [ ]:
documents = []

for i in range(1, 64):
    chunkDoc = ""
    chunkFileName = "p" + str(i) + ".txt"
    with open("data/quydinhdaotaothacsi/" + chunkFileName, "r", encoding="utf-8") as f:
        chunkDoc = f.read()
    documents.append(chunkDoc)    
    chunkDocProcessed = process_text(text=chunkDoc)
    graph.add_graph_documents(
    chunkDocProcessed,
    baseEntityLabel=True,
    include_source=True
)
    
print(len(documents))

In [ ]:
MAX_WORKERS = 2
NUM_CHUNK = 63

def process_chunk_file(i):
    chunkFileName = f"p{i}.txt"
    file_path = f"data/quydinhdaotaothacsi/{chunkFileName}"
    with open(file_path, "r", encoding="utf-8") as f:
        chunkDoc = f.read()
    return process_text(text=chunkDoc)

graph_documents = []

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = [
        executor.submit(process_chunk_file, i)
        for i in range(1, NUM_CHUNK + 1)
    ]

    for future in tqdm(
        as_completed(futures), total=len(futures), desc="Processing documents"
    ):
        chunkDocProcessed = future.result()
        graph_documents.extend(chunkDocProcessed)

graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [ ]:
entity_dist = graph.query(
    """
MATCH (d:Document)
RETURN d.text AS text,
       count {(d)-[:MENTIONS]->()} AS entity_count
"""
)
entity_dist_df = pd.DataFrame.from_records(entity_dist)
entity_dist_df["token_count"] = [
    num_tokens_from_string(str(el)) for el in entity_dist_df["text"]
]
# Scatter plot with regression line
sns.lmplot(
    x="token_count",
    y="entity_count",
    data=entity_dist_df, 
    line_kws={"color": "red"}
)
plt.title("Entity Count vs Token Count Distribution")
plt.xlabel("Token Count")
plt.ylabel("Entity Count")
plt.show()

In [ ]:
degree_dist = graph.query(
    """
MATCH (e:__Entity__)
RETURN count {(e)-[:!MENTIONS]-()} AS node_degree
"""
)
degree_dist_df = pd.DataFrame.from_records(degree_dist)

# Calculate mean and median
mean_degree = np.mean(degree_dist_df['node_degree'])
percentiles = np.percentile(degree_dist_df['node_degree'], [25, 50, 75, 90])
# Create a histogram with a logarithmic scale
plt.figure(figsize=(12, 6))
sns.histplot(degree_dist_df['node_degree'], bins=50, kde=False, color='blue')
# Use a logarithmic scale for the x-axis
plt.yscale('log')
# Adding labels and title
plt.xlabel('Node Degree')
plt.ylabel('Count (log scale)')
plt.title('Node Degree Distribution')
# Add mean, median, and percentile lines
plt.axvline(mean_degree, color='red', linestyle='dashed', linewidth=1, label=f'Mean: {mean_degree:.2f}')
plt.axvline(percentiles[0], color='purple', linestyle='dashed', linewidth=1, label=f'25th Percentile: {percentiles[0]:.2f}')
plt.axvline(percentiles[1], color='orange', linestyle='dashed', linewidth=1, label=f'50th Percentile: {percentiles[1]:.2f}')
plt.axvline(percentiles[2], color='yellow', linestyle='dashed', linewidth=1, label=f'75th Percentile: {percentiles[2]:.2f}')
plt.axvline(percentiles[3], color='brown', linestyle='dashed', linewidth=1, label=f'90th Percentile: {percentiles[3]:.2f}')
# Add legend
plt.legend()
# Show the plot
plt.show()

In [ ]:
graph.query("""
MATCH (n:`__Entity__`)
RETURN "node" AS type,
       count(*) AS total_count,
       count(n.description) AS non_null_descriptions
UNION ALL
MATCH (n)-[r:!MENTIONS]->()
RETURN "relationship" AS type,
       count(*) AS total_count,
       count(r.description) AS non_null_descriptions
""")

### De-duplication

In [ ]:
vector = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    node_label='__Entity__',
    text_node_properties=['id', 'description'],
    embedding_node_property='embedding'
)

In [ ]:
gds = GraphDataScience(
    os.environ["NEO4J_URI"],
    auth=(os.environ["NEO4J_USERNAME"], os.environ["NEO4J_PASSWORD"])
)

In [ ]:
G, result = gds.graph.project(
    "entities",                   # Graph name
    "__Entity__",                 # Node projection
    "*",                          # Relationship projection
    nodeProperties=["embedding"]  # Configuration parameters
)

In [ ]:
similarity_threshold = 0.95

gds.knn.mutate(
  G,
  nodeProperties=['embedding'],
  mutateRelationshipType= 'SIMILAR',
  mutateProperty= 'score',
  similarityCutoff=similarity_threshold
)

In [ ]:
gds.wcc.write(
    G,
    writeProperty="wcc",
    relationshipTypes=["SIMILAR"]
)

In [ ]:
word_edit_distance = 3
potential_duplicate_candidates = graph.query(
    """MATCH (e:`__Entity__`)
    WHERE size(e.id) > 3 // longer than 3 characters
    WITH e.wcc AS community, collect(e) AS nodes, count(*) AS count
    WHERE count > 1
    UNWIND nodes AS node
    // Add text distance
    WITH distinct
      [n IN nodes WHERE apoc.text.distance(toLower(node.id), toLower(n.id)) < $distance 
                  OR node.id CONTAINS n.id | n.id] AS intermediate_results
    WHERE size(intermediate_results) > 1
    WITH collect(intermediate_results) AS results
    // combine groups together if they share elements
    UNWIND range(0, size(results)-1, 1) as index
    WITH results, index, results[index] as result
    WITH apoc.coll.sort(reduce(acc = result, index2 IN range(0, size(results)-1, 1) |
            CASE WHEN index <> index2 AND
                size(apoc.coll.intersection(acc, results[index2])) > 0
                THEN apoc.coll.union(acc, results[index2])
                ELSE acc
            END
    )) as combinedResult
    WITH distinct(combinedResult) as combinedResult
    // extra filtering
    WITH collect(combinedResult) as allCombinedResults
    UNWIND range(0, size(allCombinedResults)-1, 1) as combinedResultIndex
    WITH allCombinedResults[combinedResultIndex] as combinedResult, combinedResultIndex, allCombinedResults
    WHERE NOT any(x IN range(0,size(allCombinedResults)-1,1)
        WHERE x <> combinedResultIndex
        AND apoc.coll.containsAll(allCombinedResults[x], combinedResult)
    )
    RETURN combinedResult
    """, params={'distance': word_edit_distance})

In [ ]:
system_prompt = """Xác định các thực thể trùng lặp trong danh sách và quyết định thực thể nào trong số chúng nên được hợp nhất.
Các thực thể có thể hơi khác nhau về định dạng hoặc nội dung, nhưng về cơ bản đều đề cập đến cùng một thứ. Sử dụng các kỹ năng phân tích của bạn để xác định các bản sao.

Sau đây là các quy tắc để xác định các bản sao:
1. Các thực thể có sự khác biệt nhỏ về kiểu chữ nên được coi là bản sao.
2. Các thực thể có định dạng khác nhau nhưng cùng nội dung nên được coi là bản sao.
3. Các thực thể đề cập đến cùng một đối tượng hoặc khái niệm trong thế giới thực, ngay cả khi được mô tả khác nhau, nên được coi là bản sao.
4. Nếu nó đề cập đến các số, ngày hoặc sản phẩm khác nhau, thì không được hợp nhất kết quả
"""
user_template = """
Dưới đây là danh sách các thực thể cần xử lý:
{entities}

Xác định các mục trùng lặp, hợp nhất chúng và cung cấp danh sách đã hợp nhất.
"""

In [ ]:
class DuplicateEntities(BaseModel):
    entities: List[str] = Field(
        description="Các thực thể đại diện cho cùng một đối tượng hoặc thực thể trong thế giới thực và cần được hợp nhất"
    )


class Disambiguate(BaseModel):
    merge_entities: Optional[List[DuplicateEntities]] = Field(
        description="Danh sách các thực thể đại diện cho cùng một đối tượng hoặc thực thể trong thế giới thực và cần được hợp nhất"
    )


extraction_llm = ChatOpenAI(model_name="gpt-4o").with_structured_output(
    Disambiguate
)

extraction_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system_prompt,
        ),
        (
            "human",
            user_template,
        ),
    ]
)

In [ ]:
extraction_chain = extraction_prompt | extraction_llm

def entity_resolution(entities: List[str]) -> Optional[List[List[str]]]:
    return [
        el.entities
        for el in extraction_chain.invoke({"entities": entities}).merge_entities
    ]

In [ ]:
merged_entities = []
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    # Submitting all tasks and creating a list of future objects
    futures = [
        executor.submit(entity_resolution, el['combinedResult'])
        for el in potential_duplicate_candidates
    ]

    for future in tqdm(
        as_completed(futures), total=len(futures), desc="Processing documents"
    ):
        to_merge = future.result()
        if to_merge is not None:
            merged_entities.extend(to_merge)

In [ ]:
print(merged_entities)

In [ ]:
graph.query("""
UNWIND $data AS candidates
CALL {
  WITH candidates
  MATCH (e:__Entity__) WHERE e.id IN candidates
  RETURN collect(e) AS nodes
}
CALL apoc.refactor.mergeNodes(nodes, {properties: {
    description:'combine',
    `.*`: 'discard'
}})
YIELD node
RETURN count(*)
""", params={"data": merged_entities})

# Constructing and Summarizing Communities

In [ ]:
G, result = gds.graph.project(
    "communities",  #  Graph name
    "__Entity__",  #  Node projection
    {
        "_ALL_": {
            "type": "*",
            "orientation": "UNDIRECTED",
            "properties": {"weight": {"property": "*", "aggregation": "COUNT"}},
        }
    },
)

In [ ]:
wcc = gds.wcc.stats(G)
print(f"Component count: {wcc['componentCount']}")
print(f"Component distribution: {wcc['componentDistribution']}")

In [ ]:
gds.leiden.write(
    G,
    writeProperty="communities",
    includeIntermediateCommunities=True,
    relationshipWeightProperty="weight",
)

In [ ]:
graph.query("""
MATCH (e:`__Entity__`)
UNWIND range(0, size(e.communities) - 1 , 1) AS index
CALL {
  WITH e, index
  WITH e, index
  WHERE index = 0
  MERGE (c:`__Community__` {id: toString(index) + '-' + toString(e.communities[index])})
  ON CREATE SET c.level = index
  MERGE (e)-[:IN_COMMUNITY]->(c)
  RETURN count(*) AS count_0
}
CALL {
  WITH e, index
  WITH e, index
  WHERE index > 0
  MERGE (current:`__Community__` {id: toString(index) + '-' + toString(e.communities[index])})
  ON CREATE SET current.level = index
  MERGE (previous:`__Community__` {id: toString(index - 1) + '-' + toString(e.communities[index - 1])})
  ON CREATE SET previous.level = index - 1
  MERGE (previous)-[:IN_COMMUNITY]->(current)
  RETURN count(*) AS count_1
}
RETURN count(*)
""")

In [ ]:
graph.query("""
MATCH (c:__Community__)<-[:IN_COMMUNITY*]-(:__Entity__)<-[:MENTIONS]-(d:Document)
WITH c, count(distinct d) AS rank
SET c.community_rank = rank;
""")

In [ ]:
community_size = graph.query(
    """
MATCH (c:__Community__)<-[:IN_COMMUNITY*]-(e:__Entity__)
WITH c, count(distinct e) AS entities
RETURN split(c.id, '-')[0] AS level, entities
"""
)
community_size_df = pd.DataFrame.from_records(community_size)
percentiles_data = []
for level in community_size_df["level"].unique():
    subset = community_size_df[community_size_df["level"] == level]["entities"]
    num_communities = len(subset)
    percentiles = np.percentile(subset, [25, 50, 75, 90, 99])
    percentiles_data.append(
        [
            level,
            num_communities,
            percentiles[0],
            percentiles[1],
            percentiles[2],
            percentiles[3],
            percentiles[4],
            max(subset)
        ]
    )

# Create a DataFrame with the percentiles
percentiles_df = pd.DataFrame(
    percentiles_data,
    columns=[
        "Level",
        "Number of communities",
        "25th Percentile",
        "50th Percentile",
        "75th Percentile",
        "90th Percentile",
        "99th Percentile",
        "Max"
    ],
)
percentiles_df

In [ ]:
community_info = graph.query("""
MATCH (c:`__Community__`)<-[:IN_COMMUNITY*]-(e:__Entity__)
WHERE c.level IN [0,1,4]
WITH c, collect(e ) AS nodes
WHERE size(nodes) > 1
CALL apoc.path.subgraphAll(nodes[0], {
 whitelistNodes:nodes
})
YIELD relationships
RETURN c.id AS communityId, 
       [n in nodes | {id: n.id, description: n.description, type: [el in labels(n) WHERE el <> '__Entity__'][0]}] AS nodes,
       [r in relationships | {start: startNode(r).id, type: type(r), end: endNode(r).id, description: r.description}] AS rels
""")

In [ ]:
community_template = """Dựa trên các nút và mối quan hệ được cung cấp thuộc về cùng một cộng đồng đồ thị, tạo bản tóm tắt bằng ngôn ngữ tự nhiên về thông tin được cung cấp:
{community_info}

Tóm tăt:"""  # noqa: E501

community_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Với một bộ ba đầu vào, tạo ra bản tóm tắt thông tin. Không có phần mở đầu.",
        ),
        ("human", community_template),
    ]
)

community_chain = community_prompt | llm | StrOutputParser()

In [ ]:
def prepare_string(data):
    nodes_str = "Nodes are:n"
    for node in data['nodes']:
        node_id = node['id']
        node_type = node['type']
        if 'description' in node and node['description']:
            node_description = f", description: {node['description']}"
        else:
            node_description = ""
        nodes_str += f"id: {node_id}, type: {node_type}{node_description}n"

    rels_str = "Relationships are:n"
    for rel in data['rels']:
        start = rel['start']
        end = rel['end']
        rel_type = rel['type']
        if 'description' in rel and rel['description']:
            description = f", description: {rel['description']}"
        else:
            description = ""
        rels_str += f"({start})-[:{rel_type}]->({end}){description}n"

    return nodes_str + "n" + rels_str

def process_community(community):
    stringify_info = prepare_string(community)
    summary = community_chain.invoke({'community_info': stringify_info})
    return {"community": community['communityId'], "summary": summary}

In [ ]:
summaries = []
MAX_WORKERS = 2
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = {executor.submit(process_community, community): community for community in community_info}

    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing communities"):
        summaries.append(future.result())

In [ ]:
print(summaries)

In [ ]:
graph.query("""
UNWIND $data AS row
MERGE (c:__Community__ {id:row.community})
SET c.summary = row.summary
""", params={"data": summaries})

### Local Retrieval

In [ ]:
driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))

def create_fulltext_index(tx):
    query = '''
    CREATE FULLTEXT INDEX `fulltext_entity_id` 
    FOR (n:__Entity__) 
    ON EACH [n.id];
    '''
    tx.run(query)

# Function to execute the query
def create_index():
    with driver.session() as session:
        session.execute_write(create_fulltext_index)
        print("Fulltext index created successfully.")

# Call the function to create the index
try:
    create_index()
except:
    pass

# Close the driver connection
driver.close()

### Extract entities from question

In [ ]:
class Entities(BaseModel):
    """Identifying information about entities."""

    names: list[str] = Field(
        ...,
        description="Tất cả các thực thể trong văn bản",
    )

extract_entities_from_question_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Trích xuất thực thể từ văn bản.",
        ),
        (
            "human",
            "Sử dụng định dạng đã cho để trích xuất thông tin các thực thể từ những nội dung sau, trả kết quả dưới dạng danh sách, nếu là một thực thể nhiều thông tin, hãy chia nhỏ ra "
            "input: {question}",
        ),
    ]
)

entity_chain = extract_entities_from_question_prompt | llm.with_structured_output(Entities)

### Local query


In [ ]:
BASE_LOCAL_QUERY = """CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """
LOCAL_QUERY_WITH_DESSCRIPTION = """
            CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN
                node.id + ' (desc: ' + coalesce(node.description, '') + ')' + ' - ' +
                type(r) + ' -> ' +
                neighbor.id + ' (desc: ' + coalesce(neighbor.description, '') + ')' AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN
                neighbor.id + ' (desc: ' + coalesce(neighbor.description, '') + ')' + ' - ' +
                type(r) + ' -> ' +
                node.id + ' (desc: ' + coalesce(node.description, '') + ')' AS output
            }
            RETURN output LIMIT 100
            """

LOCAL_QUERY_WITH_DESSCRIPTION_AND_SUMMARY = """
            CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})
            YIELD node, score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN
                node.id + ' (desc: ' + coalesce(node.description, '') + ')' + ' - ' +
                type(r) + ' -> ' +
                neighbor.id + ' (desc: ' + coalesce(neighbor.description, neighbor.summary, '') + ')' AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN
                neighbor.id + ' (desc: ' + coalesce(neighbor.description, neighbor.summary, '') + ')' + ' - ' +
                type(r) + ' -> ' +
                node.id + ' (desc: ' + coalesce(node.description, '') + ')' AS output
            }
            RETURN output LIMIT 50
            """

In [ ]:
def generate_full_text_query(input: str) -> str:
    words = [el for el in remove_lucene_chars(input).split() if el]
    if not words:
        return ""
    full_text_query = " AND ".join([f"{word}~2" for word in words])
    print(f"Generated Query: {full_text_query}")
    return full_text_query.strip()


# Fulltext index query
def graph_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke(question)
    for entity in entities.names:
        response = graph.query(
            LOCAL_QUERY_WITH_DESSCRIPTION,
            {"query": entity},
        )
        result += "\n".join([el['output'] for el in response])
    return result


In [ ]:
entity_chain.invoke("Đăng ký môn học?")

In [ ]:
print(graph_retriever("Điều kiện đăng ký môn học là gì?"))

In [ ]:
print(graph_retriever("Thạc sĩ nghiên cứu cần có bài báo như thế nào để đủ điều kiện bảo vệ luận văn?"))

In [ ]:
print(graph_retriever("Học viên có thể tạm dừng học bao lâu?"))

In [ ]:
# Sử dụng OpenAIEmbeddings
# embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")`
embeddings = OpenAIEmbeddings()

vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

vector_retriever = vector_index.as_retriever()

In [ ]:
def full_retriever(question: str):
    graph_data = graph_retriever(question)
    vector_data = [el.page_content for el in vector_retriever.invoke(question)]
    final_data = f"""Graph data:
{graph_data}
vector data:
{"#Document ". join(vector_data)}
    """
    return final_data

In [ ]:
template = """Trả lời câu hỏi dựa trên ngữ cảnh được cung cấp:
{context}
Thông tin trong desc là mô tả của thực thể trong ngữ cảnh. Có một số quan hệ được mô tả bằng tiếng Anh, hãy thêm chúng vào.
Câu hỏi: {question}
Sử dụng ngôn ngữ tự nhiên và trả lời ngắn gọn.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
        {
            "context": full_retriever,
            "question": RunnablePassthrough(),
        }
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
chain.invoke(input="khi nào học viên bị cảnh cáo học vụ?")

In [ ]:
chain.invoke(input="Điều kiện đăng ký môn học là gì?")

In [ ]:
chain.invoke(input="Học viên có thể tạm dừng học bao lâu?")

In [ ]:
chain.invoke(input="Thạc sĩ nghiên cứu cần có bài báo như thế nào để đủ điều kiện bảo vệ luận văn?")

### Global Retrieval


In [ ]:
def retrieve_from_community(question: str):
    # 1. Trích thực thể
    entities = entity_chain.invoke(question).names
    if not entities:
        return "No entities found."

    # 2. Tìm entity node + community
    response = graph.query("""
    CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:1})
    YIELD node RETURN node.id as entity_id
    """, {"query": entities[0]})

    if not response:
        return "Entity not found in graph."

    entity_id = response[0]['entity_id']

    # 3. Truy community id
    community = graph.query("""
    MATCH (n:__Entity__)-[:__IN_COMMUNITY__]->(c:__Community__)
    WHERE n.id = $entity_id
    RETURN c.id as community_id
    """, {"entity_id": entity_id})
    if not community:
        return "No community found."

    community_id = community[0]['community_id']

    # 4. Lấy thông tin community
    community_info = graph.query("""
    MATCH (e:__Entity__)-[:__IN_COMMUNITY__]->(:__Community__ {id: $community_id})
    OPTIONAL MATCH (e)-[r]->(n)
    RETURN e, r, n
    """, {"community_id": community_id})

    # 5. Format lại để feed vào prompt
    community_text = format_community_info(community_info)

    # 6. Tóm tắt
    summary = community_chain.invoke({"community_info": community_text})

    # 7. Trả lời
    final_answer = qa_chain.invoke({
        "context": summary,
        "question": question
    })

    return final_answer

In [ ]:
def db_query(cypher: str, params: Dict[str, Any] = {}) -> pd.DataFrame:
    """Executes a Cypher statement and returns a DataFrame"""
    return driver.execute_query(
        cypher, parameters_=params, result_transformer_=Result.to_df
    )

In [ ]:
MAP_SYSTEM_PROMPT = """
---Vai trò---

Bạn là một trợ lý hữu ích, phản hồi các câu hỏi liên quan đến dữ liệu trong các bảng được cung cấp.


---Mục tiêu---

Tạo ra một phản hồi gồm danh sách các điểm chính nhằm trả lời câu hỏi của người dùng, tóm tắt tất cả thông tin liên quan trong các bảng dữ liệu đầu vào.

Bạn nên sử dụng dữ liệu được cung cấp trong các bảng dữ liệu dưới đây làm bối cảnh chính để tạo phản hồi.
Nếu bạn không biết câu trả lời hoặc nếu các bảng dữ liệu đầu vào không chứa đủ thông tin để đưa ra câu trả lời, chỉ cần nói như vậy. Không được bịa ra bất kỳ thông tin nào.

Mỗi điểm chính trong phản hồi phải bao gồm các yếu tố sau:
- Mô tả: Một mô tả đầy đủ về điểm đó.
- Điểm quan trọng: Một số nguyên trong khoảng từ 0-100 thể hiện mức độ quan trọng của điểm đó trong việc trả lời câu hỏi của người dùng. Phản hồi kiểu “Tôi không biết” sẽ có điểm là 0.

Phản hồi phải được định dạng JSON như sau:
{{
    "points": [
        {{"description": "Mô tả điểm 1 [Dữ liệu: Báo cáo (các mã báo cáo)]", "score": giá_trị_điểm}},
        {{"description": "Mô tả điểm 2 [Dữ liệu: Báo cáo (các mã báo cáo)]", "score": giá_trị_điểm}}
    ]
}}

Phản hồi phải giữ nguyên ý nghĩa gốc và cách sử dụng các động từ tình thái như “sẽ”, “có thể” .

Các điểm có dữ liệu hỗ trợ nên liệt kê các báo cáo liên quan làm tài liệu tham khảo như sau:
"Đây là một câu ví dụ được hỗ trợ bởi các tài liệu tham khảo [Dữ liệu: Báo cáo (các mã báo cáo)]"

**Không liệt kê nhiều hơn 5 mã báo cáo trong một tài liệu tham khảo**. Thay vào đó, hãy liệt kê 5 mã báo cáo liên quan nhất và thêm “+more” để cho biết còn nhiều hơn nữa.

Ví dụ:
"Người X là chủ sở hữu của Công ty Y và bị cáo buộc nhiều sai phạm [Dữ liệu: Báo cáo (2, 7, 64, 46, 34, ...)]. Anh ta cũng là CEO của công ty X [Dữ liệu: Báo cáo (1, 3)]"

trong đó 1, 2, 3, 7, 34, 46, và 64 là mã (không phải chỉ số) của các báo cáo dữ liệu liên quan trong bảng được cung cấp.

Không bao gồm thông tin nếu không có bằng chứng hỗ trợ từ dữ liệu.

---Các bảng dữ liệu---

{context_data}
"""


map_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            MAP_SYSTEM_PROMPT,
        ),
        (
            "human",
            "{question}",
        ),
    ]
)

map_chain = map_prompt | llm | StrOutputParser()

In [ ]:
REDUCE_SYSTEM_PROMPT = """
---Vai trò---

Bạn là một trợ lý hữu ích, trả lời các câu hỏi về một tập dữ liệu bằng cách tổng hợp các quan điểm từ nhiều nhà phân tích khác nhau.


---Mục tiêu---

Tạo ra một phản hồi với độ dài và định dạng mục tiêu nhằm trả lời câu hỏi của người dùng, đồng thời tóm tắt tất cả các báo cáo từ nhiều nhà phân tích, mỗi người tập trung vào các phần khác nhau của tập dữ liệu.

Lưu ý rằng các báo cáo của nhà phân tích được cung cấp dưới đây **được sắp xếp theo thứ tự quan trọng giảm dần**.

Nếu bạn không biết câu trả lời hoặc nếu các báo cáo được cung cấp không chứa đủ thông tin để đưa ra câu trả lời, chỉ cần nói như vậy. Không được bịa ra bất kỳ thông tin nào.

Phản hồi cuối cùng nên loại bỏ tất cả thông tin không liên quan từ các báo cáo của nhà phân tích và tổng hợp thông tin đã được làm sạch thành một câu trả lời toàn diện, giải thích tất cả các điểm chính và các hàm ý, phù hợp với độ dài và định dạng của phản hồi.

Hãy thêm các phần và bình luận vào phản hồi nếu phù hợp với độ dài và định dạng. Định dạng phản hồi bằng **markdown**.

Phản hồi phải giữ nguyên ý nghĩa gốc và cách sử dụng các động từ tình thái như “sẽ”, “có thể”.

Phản hồi cũng phải giữ lại tất cả các tham chiếu dữ liệu đã được nêu trong các báo cáo của nhà phân tích, nhưng **không được đề cập đến vai trò của các nhà phân tích trong quá trình phân tích**.

**Không liệt kê nhiều hơn 5 mã báo cáo trong một tham chiếu**. Thay vào đó, hãy liệt kê 5 mã báo cáo liên quan nhất và thêm “+more” để chỉ ra rằng còn nhiều hơn nữa.

Ví dụ:

"Người X là chủ sở hữu của Công ty Y và bị cáo buộc nhiều sai phạm [Dữ liệu: Báo cáo (2, 7, 34, 46, 64, ...)]. Anh ta cũng là CEO của công ty X [Dữ liệu: Báo cáo (1, 3)]"

trong đó 1, 2, 3, 7, 34, 46 và 64 là mã (không phải chỉ số) của các báo cáo dữ liệu liên quan.

Không bao gồm thông tin nếu không có bằng chứng hỗ trợ từ dữ liệu.


---Độ dài và định dạng phản hồi mục tiêu---

{response_type}


---Báo cáo của các nhà phân tích---

{report_data}

---Độ dài và định dạng phản hồi mục tiêu---

{response_type}

Hãy thêm các phần và bình luận vào phản hồi nếu phù hợp với độ dài và định dạng. Định dạng phản hồi bằng markdown.
"""

reduce_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            REDUCE_SYSTEM_PROMPT,
        ),
        (
            "human",
            "{question}",
        ),
    ]
)
reduce_chain = reduce_prompt | llm | StrOutputParser()

In [ ]:
response_type: str = "nhiều đoạn văn bản"


def global_retriever(query: str, level: int, response_type: str = response_type) -> str:
    community_data = graph.query(
        """
    MATCH (c:__Community__)
    WHERE c.level = $level
    RETURN c.summary AS output
    """,
        params={"level": level},
    )
    intermediate_results = []
    for community in tqdm(community_data, desc="Processing communities"):
        intermediate_response = map_chain.invoke(
            {"question": query, "context_data": community["output"]}
        )
        intermediate_results.append(intermediate_response)
    final_response = reduce_chain.invoke(
        {
            "report_data": intermediate_results,
            "question": query,
            "response_type": response_type,
        }
    )
    return final_response

In [ ]:
print(global_retriever("Tóm tắt thông tin về chương trình đào tạo?", 1))

In [ ]:
community_data = graph.query(
        """
    MATCH (c:__Community__)
    WHERE c.level = $level
        RETURN c.summary AS output
    """,
        params={"level": 0},
    )

In [ ]:
for community in tqdm(community_data, desc="Processing communities"):
    print(community["output"])

In [ ]:
print(global_retriever("Tóm tắt thông tin về chương trình đào tạo?", 0))

In [ ]:
from langchain_core.messages import HumanMessage

DETERMINE_QUERY_TYPE_PROMT = """
Bạn là một trợ lý AI hỗ trợ truy vấn kiến thức từ đồ thị (GraphRAG).
Với mỗi câu hỏi tiếng Việt của người dùng, hãy phân tích và cho biết nên sử dụng "global" hay "local" query để tìm kiếm thông tin.

Trả lời duy nhất bằng một từ: "global" hoặc "local".

Với "global" là truy vấn được gửi trực tiếp đến toàn bộ các bảng tóm tắt của các cộng đồng trong đồ thị, mà không giới hạn phạm vi tìm kiếm vào một phần nhỏ cụ thể.
Mục tiêu: tìm kiếm toàn cục, thường được dùng khi chưa biết rõ ngữ cảnh hoặc muốn lấy thông tin liên quan từ toàn bộ hệ thống kiến thức.
Khi nên dùng:
Khi truy vấn ban đầu chưa có ngữ cảnh cụ thể.
Khi cần xác định vùng liên quan đến truy vấn.
Khi muốn thực hiện “zero-shot” truy xuất thông tin từ toàn bộ nguồn kiến thức.

Còn "local" là truy vấn giới hạn trong một phần con của đồ thị – ví dụ như một neighborhood xung quanh một node cụ thể.
Mục tiêu: khai thác cục bộ, tận dụng ngữ cảnh từ một phần cụ thể của đồ thị đã biết trước.
Khi nên dùng:
Khi đã xác định được “entry point” trong đồ thị (ví dụ: node tương ứng với thực thể người, tổ chức…).
Khi chỉ cần truy xuất các thông tin có liên hệ trực tiếp đến node hiện tại.
Khi đã có kết quả từ global query và muốn mở rộng hoặc đào sâu ngữ nghĩa từ khu vực đó.

Câu hỏi: {question}
"""

def determine_query_type(question: str) -> str:
    response = llm.invoke([HumanMessage(content=DETERMINE_QUERY_TYPE_PROMT)])
    return response.content.strip().lower()

In [ ]:
determine_query_type("Tóm tắt thông tin về chương trình đào tạo?")

In [ ]:
determine_query_type("Điều kiện Đăng ký môn học gồm những gì?")


### Evaluation

In [ ]:
from bert_score import score as bert_score
import string

def normalize_text(text):
    """Chuẩn hóa văn bản để so sánh exact match."""
    text = text.lower()
    text = text.strip()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

def evaluate_qa_response(prediction, ground_truth, lang='en'):
    """
    Đánh giá độ chính xác câu trả lời bằng Exact Match và BERTScore.
    
    Args:
        prediction (str): câu trả lời từ mô hình
        ground_truth (str): câu trả lời đúng
        lang (str): ngôn ngữ, mặc định là 'en' (hỗ trợ cả 'vi')
        
    Returns:
        dict: {'exact_match': ..., 'bertscore': ...}
    """
    P, R, F1 = bert_score([prediction], [ground_truth], lang=lang, rescale_with_baseline=True)
    
    return {
        'bertscore_precision': round(P[0].item(), 4), # Trung bình độ tương đồng của mỗi token trong câu trả lời mô hình với token gần nhất trong đáp án chuẩn.
        'bertscore_recall': round(R[0].item(), 4), # Trung bình độ tương đồng của mỗi token trong đáp án chuẩn với token gần nhất trong câu trả lời mô hình.
        'bertscore_f1': round(F1[0].item(), 4) # Trung bình hài hòa giữa Precision và Recall.
    }


In [ ]:
ground_truth = "Sinh viên phải hoàn thành 135 tín chỉ để tốt nghiệp."
prediction = "Để tốt nghiệp, sinh viên cần hoàn tất 135 tín chỉ."

result = evaluate_qa_response(prediction, ground_truth, lang='vi')
print(result)

In [ ]:
evaluation_promt = """
Bạn là một giám khảo chuyên đánh giá chất lượng câu trả lời từ hệ thống truy xuất thông tin trong lĩnh vực giáo dục.

Dưới đây là một câu hỏi, một câu trả lời từ mô hình, và thông tin tài liệu được truy xuất (nếu có).

Hãy đánh giá câu trả lời theo 5 tiêu chí sau:

1. Correctness (Câu trả lời có đúng thông tin không?)
2. Faithfulness (Câu trả lời có dựa đúng vào tài liệu không?)
3. Clarity (Câu trả lời có rõ ràng, dễ hiểu không?)
4. Completeness (Câu trả lời có đầy đủ ý không?)
5. Usefulness (Câu trả lời có giúp ích cho người hỏi không?)

Với mỗi tiêu chí, hãy chấm điểm từ 1 đến 5 và kèm theo nhận xét ngắn.
"""

In [ ]:
def generate_real_answers_from_csv(csv_path: str, output_path: str, chain):
    import pandas as pd

    df = pd.read_csv(csv_path)
    real_answers = []

    for i, row in df.iterrows():
        try:
            result = chain.invoke({"question": row["question"]})

            # Xử lý tuỳ vào kiểu object trả về
            if hasattr(result, "content"):  # nếu là Chat model
                real_answers.append(result.content)
            elif hasattr(result, "model_dump_json"):  # nếu là Pydantic
                real_answers.append(result.model_dump_json())
            else:
                real_answers.append(str(result))

        except Exception as e:
            real_answers.append(f"ERROR: {e}")

    df["real_answer"] = real_answers
    df.to_csv(output_path, index=False)
    return df

In [ ]:
generate_real_answers_from_csv("evaluation_data/Quy_dinh_dao_tao_Thac_si.csv", "evaluation_data/out_put_Quy_dinh_dao_tao_Thac_si.csv", chain)

In [ ]:
graph_retriever("Thời lượng tối thiểu của môn học là bao nhiêu tín chỉ?")

In [ ]:
real_answers = []
result1 = chain.invoke("Quy định đào tạo thạc sĩ được ban hành theo quyết định số mấy?")
real_answers.append(result1)
print(real_answers)

In [ ]:
graph_retriever("Điều kiện bảo vệ luận văn là gì?")

In [ ]:
chain.invoke("Điều kiện bảo vệ luận văn là gì?")

In [ ]:
entity_chain.invoke("Điều kiện bảo vệ luận văn là gì?")